In [2]:
import os
import numpy as np
import tensorflow as tf
from data_loader import data_generator
from sklearn.metrics import f1_score, precision_score, recall_score
import datetime
from helpers import f1 as f1_metric
from model import vit_without_mlp

def set_random_seeds(seed):
    """Set random seeds for reproducibility."""
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

def create_data_generators():
    """Create data generators for training and validation."""
    train_generator = data_generator(
        image_dir='data_v3_processed/train/images/images',
        mask_dir='data_v3_processed/train/masks/masks',
        batch_size=8
    )
    
    val_generator = data_generator(
        image_dir='data_v3_processed/val/images/images',
        mask_dir='data_v3_processed/val/masks/masks',
        batch_size=8
    )

    return train_generator, val_generator

def train_and_evaluate_model(run_number, train_generator, val_generator, 
                           epochs=7, steps_per_epoch=1500, validation_steps=5, 
                           random_seed=None):
    """Train a single model and return evaluation metrics."""
    print(f"\nTraining Run {run_number + 1}/5 (Seed: {random_seed})")
    print("=" * 50)
    
    if random_seed is not None:
        set_random_seeds(random_seed)
    
    model = vit_without_mlp()
    
    model.compile(optimizer='adam', 
                 loss='binary_crossentropy', 
                 metrics=[f1_metric])
    
    try:
        history = model.fit(train_generator,
                           validation_data=val_generator,
                           epochs=epochs,
                           steps_per_epoch=steps_per_epoch,
                           validation_steps=validation_steps,
                           verbose=1)

        model_path = os.path.join("vit_no_mlp", f"vit_no_mlp_run{run_number+1}.h5")
        os.makedirs("vit_no_mlp", exist_ok=True)
        model.save(model_path)
        
        return model_path
        
    except Exception as e:
        print(f"ERROR in training: {e}")
        return None

# Main execution
random_seeds = [42, 123, 456, 789, 999]

for run in range(5):
    # CREATE FRESH GENERATORS FOR EACH RUN - this fixes the state issue
    train_generator, val_generator = create_data_generators()
    
    path = train_and_evaluate_model(
        run, train_generator, val_generator, random_seed=random_seeds[run]
    )
    
    tf.keras.backend.clear_session()


Training Run 1/5 (Seed: 42)
Epoch 1/7
1500/1500 [==============================] - 59s 34ms/step - loss: 0.0154 - f1: 9.0148e-06 - val_loss: 0.0596 - val_f1: 0.0000e+00
Epoch 2/7
1500/1500 [==============================] - 51s 34ms/step - loss: 0.0078 - f1: 0.0379 - val_loss: 0.0207 - val_f1: 0.5188
Epoch 3/7
1500/1500 [==============================] - 51s 34ms/step - loss: 0.0037 - f1: 0.2610 - val_loss: 0.0195 - val_f1: 0.4939
Epoch 4/7
1500/1500 [==============================] - 51s 34ms/step - loss: 0.0031 - f1: 0.2991 - val_loss: 0.0192 - val_f1: 0.6099
Epoch 5/7
1500/1500 [==============================] - 51s 34ms/step - loss: 0.0022 - f1: 0.3094 - val_loss: 0.0147 - val_f1: 0.5865
Epoch 6/7
1500/1500 [==============================] - 51s 34ms/step - loss: 0.0021 - f1: 0.3227 - val_loss: 0.0137 - val_f1: 0.6696
Epoch 7/7
1500/1500 [==============================] - 51s 34ms/step - loss: 0.0017 - f1: 0.3223 - val_loss: 0.0166 - val_f1: 0.6671

Training Run 2/5 (Seed: 123)
Ep